In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import scipy
import scipy.stats as stats
import os

In [2]:
# project files

from utils.variables import pack, unpack
from utils.display import table, tabulate
from utils.files import download_sheet, save, load
from utils.hash import digest


from fit_options import fit_options
from game_model import game
from rp_model import compute_rp, make_precomputed_columns
from initial_guess import make_initial_guess


In [3]:
# stuff for display

from IPython.lib.pretty import pretty, pprint

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [4]:
# Load previous data

data = pd.read_pickle(fit_options.data_file)
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,FreqL,Inv,Berry1,BerryL,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,BerryD,IngD,Dupes,Amnt,Ing2P,Help skill bonus,RP Multiplier
count,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.000000,6534.0,6534.000000,6534.000000,6534.000000,6534.000000
mean,10.853229,615.533364,1.502908,615.572189,0.038825,3944.459749,3791.715011,12.276094,28.061524,38.029691,109.003979,1.022954,0.988395,0.994858,1.000612,1.005326,0.207776,33.046729,55.868228,149.126645,1.0,0.078359,3.244873,0.987198,1.028745
std,7.311007,400.372168,0.995278,400.424290,0.985871,964.776439,1027.439788,3.398884,2.615911,8.230885,16.157611,0.305288,0.059258,0.044912,0.117631,0.118704,0.044995,36.934256,28.297937,61.165970,0.0,0.533277,21.530233,0.038346,0.071271
min,1.000000,245.000000,1.000000,245.060000,-11.980000,2200.000000,1489.118400,7.000000,24.000000,24.000000,90.000000,0.520000,0.890000,0.920000,0.800000,0.800000,0.130000,5.880000,25.000000,90.000000,1.0,0.000000,0.000000,0.790000,1.000000
25%,5.000000,356.000000,1.000000,356.392500,-0.290000,3200.000000,2934.360000,10.000000,26.000000,32.000000,98.000000,0.802500,1.000000,1.000000,1.000000,1.000000,0.180000,12.810000,35.000000,101.000000,1.0,0.000000,0.000000,1.000000,1.000000
50%,10.000000,470.000000,1.000000,470.100000,-0.010000,3900.000000,3773.000000,12.000000,28.000000,36.000000,103.000000,0.950000,1.000000,1.000000,1.000000,1.000000,0.200000,20.190000,46.000000,121.000000,1.0,0.000000,0.000000,1.000000,1.000000
75%,14.000000,724.000000,2.000000,723.817500,0.270000,4500.000000,4443.750000,14.000000,31.000000,42.000000,115.000000,1.220000,1.000000,1.000000,1.000000,1.000000,0.240000,43.980000,68.000000,202.000000,1.0,0.000000,0.000000,1.000000,1.000000
max,42.000000,3479.000000,6.000000,3479.540000,10.460000,6300.000000,6916.140000,23.000000,35.000000,83.000000,151.000000,2.410000,1.120000,1.080000,1.200000,1.200000,0.420000,413.950000,243.000000,511.000000,1.0,8.000000,342.000000,1.000000,1.440000


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
0,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.42,0.42,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,3700.0,3276.72,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,1.09,0.90,1.00,1.0,0.8,0.26,22.76,538.36,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
1,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,530.70,-0.30,Neutral,ArbokNeutral81,-,-,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.0,1.0,0.26,22.76,531.00,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
2,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.09,0.09,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.2,1.0,0.26,22.76,530.61,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
3,Arcanine,5.0,958.0,Calm,3.0,,957.90,-0.10,Calm,ArcanineCalm53,Main Skill Chance,Speed of Help,2500.0,2728.00,Fire,Skills,Extra Helpful S,16.0,27.0,31,Fiery Herb,130.0,1.31,1.11,1.00,1.0,1.2,0.14,84.50,937.84,ArcanineNeutral53,31.0,130.0,1,True,,,,0.0,0.0,1.0,1.0
4,Arcanine,3.0,715.0,Lax,2.0,pokemon sleep general,714.77,-0.23,Lax,ArcanineLax32,Energy Recovery,Main Skill Chance,2500.0,2490.00,Fire,Skills,Extra Helpful S,16.0,27.0,29,Fiery Herb,130.0,1.44,1.00,1.08,1.0,0.8,0.14,61.21,750.21,ArcanineNeutral32,29.0,130.0,1,True,,,,0.0,0.0,1.0,1.0


In [5]:
# Load previous fit

x0, unpack_info = pack(make_initial_guess())

hash_value = digest(data, x0)
filename = fit_options.result_file(hash_value)

opt = load(filename)
sol = unpack(opt.x, unpack_info)

opt

    message: `ftol` termination condition is satisfied.
    success: True
     status: 2
          x: [ 2.563e-01  2.544e-01 ...  2.212e-01  2.211e-01]
       cost: 895.6192997918448
       grad: [-1.035e-05 -1.478e-05 ...  0.000e+00  0.000e+00]
 optimality: 0.2867610949324444
       nfev: 39
       njev: 23

In [6]:
# Helpers

def truncated_normal_sample(size, mu, sigma, lower, upper):
    return stats.truncnorm.rvs( (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma, size=size)

def round_noise(size):
    eps = 1e-6
    return truncated_normal_sample(size, mu=0.0, sigma=0.25, lower= -0.5+eps, upper= 0.5-eps)

def harmonic(a,b):
    return 2.0 / ( 1.0 / a + 1.0 / b )

In [7]:
# In the boostrap method,
# We create N synthetic datasets that are likely to represent the population
# Then we redo the fit on those, starting from the best fit in initial dataset.
# Finally, we collect statistics on the fitted variables between runs

# To create the synthetic datasets, we use sampling with replacement (allow duplicate)
# Because some Pokémon are rare, we use stratified sampling

stratas = data.groupby(['Pokemon'], group_keys=False)
target_group_size =  stratas.size().quantile(q=0.75)
    

n_boostraps = 50
all_opt_x = [opt.x]

In [8]:
cache_fit = True
boostrap_filename = f"./results/bootstrap-fit-{hash_value}.pickle"

if cache_fit and os.path.isfile(boostrap_filename):

    all_opt_x = load(boostrap_filename)
    print("Loaded from cache")

else:

    # WARNING: RUNNING THE OPTIMISATION N TIMES IS VERY LONG.
    #
    # YOU CAN INTERRUPT THE KERNEL (STOP BUTTON)
    # AND RUN THE NEXT CELLS IF YOU WANT TO SEE CURRENT PROGRESS
    #
    # THE COLLECTION `all_opt_x` IS DEFINED ABOVE THIS CELL
    # SO YOU CAN RESUME ADDING MORE RUNS

    while( len(all_opt_x) < n_boostraps ):

        print( "Boostrap run " + str(len(all_opt_x)) )

        # To create the synthetic datasets, we use sampling with replacement (allow duplicate)

        # We'll introduce a correction toward more equal sample size between Pokémon
        # We can motivate that by the fact stratified sampling should use population proportions
        # instead of current data proportions

        resampled = stratas.apply(lambda x: x.sample( round( harmonic(len(x),target_group_size) ), replace=True, ignore_index=True))

        # Add small noise to RP
        # We do so to simulate some unknown value that would round() to current RP

        referenceRP = resampled["RP"].to_numpy()
        referenceRP += round_noise(len(resampled))

        # Compute per sample information about help time, nature, sub-skills etc
        recomputed = make_precomputed_columns(resampled)

        # Put the pieces together

        def residual(x):
            return referenceRP - compute_rp(x, resampled, recomputed, unpack_info)

        #fit_options.soft_round.exact = False
        #fit_options.soft_round.alpha = 6

        # Redo the fit, starting from the optimal we found on current data
        opt2 = scipy.optimize.least_squares(residual, opt.x, **fit_options.least_squares_kwargs)

        # Collect results for stats
        all_opt_x.append(opt2.x)

    # Save
    save(boostrap_filename, all_opt_x)

Boostrap run 1
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         9.3457e+02                                    1.08e+07    
       1              7         8.9878e+02      3.58e+01       3.33e-01       2.41e+04    
       2             10         8.9871e+02      6.70e-02       4.16e-02       2.02e+02    
       3             12         8.9869e+02      1.62e-02       2.08e-02       7.59e+01    
       4             13         8.9868e+02      1.26e-02       4.16e-02       7.73e+01    
       5             16         8.9868e+02      3.51e-04       5.20e-03       4.73e+00    
       6             18         8.9868e+02      9.38e-05       2.60e-03       1.15e+00    
       7             20         8.9868e+02      2.81e-05       1.30e-03       2.45e-01    
       8             22         8.9868e+02      9.52e-06       6.50e-04       3.48e-02    
       9             23         8.9868e+02      1.02e-05       1.30e-03    

In [10]:
# Stats
# We'll use the median as estimate for the mean.
# And 1.4826 * < median absolute deviation from the median > as estimate for std

center = np.median(all_opt_x, axis=0)

interval_std = 1.96 * np.std( all_opt_x, axis=0) # 95% confidence region
interval_mad = 3 * np.median( list(map(lambda x: np.abs(center-x), all_opt_x)), axis=0) # 95% confidence region

sol_center = unpack(center, unpack_info)
sol_interval_std = unpack(interval_std, unpack_info)
sol_interval_mad = unpack(interval_mad, unpack_info)

# Pretty display
pd.DataFrame({
    
    "Pokemon":game.data.pokedex["Pokemon"],
    
    "ing%": sol_center["Pokemons ing fractions"]*100.0,
    #"conf (ing)": sol_interval_std["Pokemons ing fractions"]*100.0,
    "conf (ing)*": sol_interval_mad["Pokemons ing fractions"]*100.0,

    "skill% * skillValue": sol_center["Pokemons skill products"], 
    #"conf (skill)": sol_interval_std["Pokemons skill products"],
    "conf (skill)*": sol_interval_mad["Pokemons skill products"],


}).set_index("Pokemon")

,ing%,conf (ing)*,skill% * skillValue,conf (skill)*
Pokemon,,,,
Bulbasaur,25.624110,0.047530,16.703848,0.053252
Ivysaur,25.428206,0.026336,16.709465,0.023047
Venusaur,26.548789,0.022579,18.467881,0.025624
Charmander,20.038437,0.047952,9.672995,0.075889
Charmeleon,22.660104,0.025697,14.048389,0.026762
Charizard,22.340223,0.046482,14.113455,0.072614
Squirtle,27.008765,0.040366,17.597913,0.062402
Wartortle,27.029689,0.030222,17.583056,0.021457
Blastoise,27.420885,0.031373,18.455569,0.017314


In [ ]:
# Explain what we did with the re-sampling target size
# pd.DataFrame({'before': stratas.size(), 'after': stratas.apply(lambda x: round( harmonic(len(x), target_group_size) )) })